In [32]:
import pandas as pd
import numpy as np

In [33]:
import sklearn
import seaborn as sns

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [40]:
df = pd.read_csv("data/train.csv")

In [41]:
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [42]:
df.Gender = df.Gender.map({"Male":0,"Female":1})
df.Vehicle_Age = df.Vehicle_Age.map({"< 1 Year":0,"1-2 Year":1,"> 2 Years":2})
df.Vehicle_Damage = df.Vehicle_Damage.map({"No":0,"Yes":1})

df.Region_Code[df.Region_Code != 28.0] = 0
df.Region_Code[df.Region_Code == 28.0] = 1

df.Policy_Sales_Channel[df.Policy_Sales_Channel != 152.0] = 0
df.Policy_Sales_Channel[df.Policy_Sales_Channel == 152.0] = 1

<ipython-input-42-e44f2e74751d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Region_Code[df.Region_Code != 28.0] = 0
<ipython-input-42-e44f2e74751d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Region_Code[df.Region_Code == 28.0] = 1
<ipython-input-42-e44f2e74751d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Policy_Sales_Channel[df.Policy_Sales_Channel != 152.0] = 0
<ipython-input-42-e44f2e74751d>:9: SettingWithCo

In [43]:
df.shape

(381109, 12)

In [44]:
q = df.Annual_Premium.quantile(0.99)
df = df[df.Annual_Premium < q]

In [45]:
df.shape

(377296, 12)

In [46]:
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,0,44,1,1.0,0,2,1,40454.0,0.0,217,1
1,2,0,76,1,0.0,0,1,0,33536.0,0.0,183,0
2,3,0,47,1,1.0,0,2,1,38294.0,0.0,27,1
3,4,0,21,1,0.0,1,0,0,28619.0,1.0,203,0
4,5,1,29,1,0.0,1,0,0,27496.0,1.0,39,0


In [48]:
df_X = df[["Gender","Age","Driving_License","Region_Code","Previously_Insured","Vehicle_Age","Vehicle_Damage","Annual_Premium","Policy_Sales_Channel","Vintage"]]

In [49]:
df_X.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,0,44,1,1.0,0,2,1,40454.0,0.0,217
1,0,76,1,0.0,0,1,0,33536.0,0.0,183
2,0,47,1,1.0,0,2,1,38294.0,0.0,27
3,0,21,1,0.0,1,0,0,28619.0,1.0,203
4,1,29,1,0.0,1,0,0,27496.0,1.0,39


In [50]:
sc = StandardScaler()
df_X = sc.fit_transform(df_X)

In [64]:
print(df_X)

[[-0.9220567   0.33713602  0.04600947 ...  0.68452898 -0.74097932
   0.74887115]
 [-0.9220567   2.403049    0.04600947 ...  0.23549148 -0.74097932
   0.3424931 ]
 [-0.9220567   0.53081536  0.04600947 ...  0.54432646 -0.74097932
  -1.52206502]
 ...
 [-0.9220567  -1.14773894  0.04600947 ...  0.33817684 -0.74097932
   0.07954259]
 [ 1.08453201  1.88657075  0.04600947 ...  0.95474336 -0.74097932
  -0.96030712]
 [-0.9220567   0.46625558  0.04600947 ...  0.77040302 -0.74097932
   0.98791706]]


In [55]:
df_Y = df.Response.to_numpy()

In [63]:
df_Y

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [66]:
X_train,X_test,y_train,y_test = train_test_split(df_X,df_Y,test_size=.20,random_state=1,stratify=df_Y)

In [72]:
train = pd.concat([pd.DataFrame(y_train),pd.DataFrame(X_train)],axis = 1)
test = pd.concat([pd.DataFrame(y_test),pd.DataFrame(X_test)],axis = 1)

In [79]:
test.to_csv("processed_data/validation.csv", header=False, index=False)
train.to_csv("processed_data/train.csv", header=False, index=False)

In [ ]:
Copiar el proyecto github.

Copiarlo en sagemaker, crear un nuevo cuaderno en el cual se va a subir la información a s3
en el nuevo cuaderno también se va a especificar el modelo y a correr según el ejemplo de imdb con hyperparameter tunning 
obtener el mejor modelo
evaluar si hacer el trabajo escrito o implementar una aplicacion web